# Combining Image Classification and NLP Models Together

## Import Libraries

In [1]:
# change the python's path to the parent directory
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import numpy as np

from models.popular_models import PopularModels
from models.lstm import LSTM
from torchvision import transforms
from tqdm import tqdm
from data.dataset import MultiLabelDataset
from tools.tools import get_data, load_data, tokenize, remove_class, count_class, calculate_pos_weights
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.model_selection import train_test_split
from typing import Tuple, Any

/home/reynardo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Remove unused memory in GPU

In [3]:
gc.collect()
torch.cuda.empty_cache()

## Load the Data

In [4]:
# import data
train_data = get_data("../dataset/train.csv")
test_data = get_data("../dataset/test.csv")

# perform text cleaning and get the pandas' dataframe
train_data = load_data(train_data)
test_data = load_data(test_data, has_label=False)

# remove an imbalanced class
train_data = remove_class(train_data, class_no=1)

# split into training and validating sets
X = train_data.iloc[:, 0:2]
y = train_data.iloc[:, 2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_test, y_test], axis=1)

# join the data together
for_nlp_data = pd.concat((train_data['caption'], val_data['caption'], test_data['caption']), ignore_index=True)

In [5]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(12740, 21)
(3185, 21)
(10000, 2)


In [6]:
print(f"Number of training instances: 	{train_data.shape[0]}")
print(f"Number of validation instances: {val_data.shape[0]}")
print(f"Number of testing instances:  	{test_data.shape[0]}")

Number of training instances: 	12740
Number of validation instances: 3185
Number of testing instances:  	10000


## Preprocessing for Images and Caption

In [7]:
# define the image transformation: currently following resnet18
transform = transforms.Compose([
	transforms.Resize((232, 232)),
	transforms.CenterCrop(224),
	transforms.ToTensor(), # converts images to [0, 1]
	transforms.Normalize(
		mean=[0.485, 0.456, 0.406],
		std=[0.229, 0.224, 0.225],
	)
])

# tokenize the data
final_list, vocab = tokenize(for_nlp_data)
X_train_vec = final_list[:train_data.shape[0], :]
X_val_vec = final_list[train_data.shape[0]:train_data.shape[0]+val_data.shape[0], :]
X_test_vec = final_list[train_data.shape[0]+val_data.shape[0]:, :]

print(final_list.shape)
print(X_train_vec.shape)
print(X_val_vec.shape)
print(X_test_vec.shape)

Max Sentence Length: 28
(25925, 28)
(12740, 28)
(3185, 28)
(10000, 28)


## Create Dataset and DataLoader

In [8]:
# initialize the dataset
train_dataset = MultiLabelDataset(
	csv_file=train_data,
	root_dir='../dataset/data/',
	vectorizer=None,
	transform=transform,
	use_caption_vec=True,
	caption_vec=X_train_vec,
	is_test=False,
)
val_dataset = MultiLabelDataset(
    csv_file=val_data,
    root_dir='../dataset/data/',
    vectorizer=None,
	transform=transform,
	use_caption_vec=True,
	caption_vec=X_val_vec,
	is_test=False,
)
test_dataset = MultiLabelDataset(
	csv_file=test_data,
	root_dir='../dataset/data/',
	vectorizer=None,
	transform=transform,
	use_caption_vec=True,
	caption_vec=X_test_vec,
	is_test=True,
)

BATCH_SIZE=16

# load the dataset into batches 
train_dataloader = DataLoader(
	dataset=train_dataset,
	batch_size=BATCH_SIZE,
	shuffle=True,
)
val_dataloader = DataLoader(
    dataset=val_dataset,
	batch_size=BATCH_SIZE,
	shuffle=True,
)
test_dataloader = DataLoader(
	dataset=test_dataset,
	batch_size=BATCH_SIZE,
	shuffle=False,
)

## Create the Combined Model

In [9]:
class CombinedModel(nn.Module):

	def __init__(
			self, 
			choice: str, 
			pretrained: bool, 
			freeze: bool, 
			cnn_n_out: int,
			no_layers: int,
			vocab_size: int,
			embedding_dim: int,
			lstm_hidden_dim: int,
			lstm_n_out: int,
			lstm_dropout: float = 0.5,
			fc_dropout: float = 0.5,
		) -> None:
		super(CombinedModel, self).__init__()

		# get the CNN model for image classification
		self.cnn_model = PopularModels(
			choice=choice,
			pretrained=pretrained,
			freeze=freeze,
			n_out=cnn_n_out,
		).get_model()

		# get the LSTM model for text classification
		self.lstm_model = LSTM(
			no_layers=no_layers,
			vocab_size=vocab_size + 1,
			embedding_dim=embedding_dim,
			hidden_dim=lstm_hidden_dim,
			output_dim=lstm_n_out,
			dropout=lstm_dropout,
		)
		
		# create the final fully connected layer
		self.last_layer = nn.Linear(cnn_n_out + lstm_n_out, 19)
		
		# prevent overfitting
		self.dropout_layer = nn.Dropout(p=fc_dropout, inplace=True)

	def forward(self, x, y, hidden):
		x = self.cnn_model(x)
		y, hidden = self.lstm_model(y, hidden)
		
		output = torch.cat((x, y), dim=1)
		output = self.dropout_layer(output)
		output = self.last_layer(output)

		return output, hidden

## Get the Positive Weights (Data Imbalance Strategy)

In [10]:
class_counts = list(count_class(train_data).values())
pos_weights = calculate_pos_weights(class_counts, train_data)
if torch.cuda.is_available():
	pos_weights = pos_weights.to('cuda')

## Define training function

In [11]:
def train_and_val(model: CombinedModel,
				  epochs: int,
				  n_classes: int,
				  optimizer: torch.optim.Optimizer,
				  loss_fn: nn.Module,
				  threshold: int,
				  batch_size: int) -> Tuple[CombinedModel, dict[str, Any]]:

	clip = 5 # for exploding gradient problem
	train_losses = []
	sk_val_accs = []
	sk_train_accs = []
	sk_val_precisions = []
	sk_train_precisions = []
	sk_val_recalls = []
	sk_train_recalls = []
	sk_val_f1_scores = []
	sk_train_f1_scores = []
	for epoch in range(epochs):

		# initialize the hidden state
		hidden = model.lstm_model.init_hidden(batch_size=batch_size)

		start = 0 # index
		y_true = np.zeros((train_data.shape[0], n_classes))
		y_preds = np.zeros((train_data.shape[0], n_classes))
		train_loss = 0.
		model.train()
		for _, images, captions, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training: "):

			if torch.cuda.is_available():
				images = images.to('cuda')
				captions = captions.to('cuda')
				labels = labels.to('cuda')
			
			# creating new variables for the hidden state, otherwise
			# we'd backprop through the entire training history
			hidden = tuple([each.data for each in hidden])

			optimizer.zero_grad()
			y_pred, hidden = model(images, captions, hidden)

			# backward
			loss = loss_fn(y_pred, labels)
			loss.backward()

			# clip_grad_norm helps prevent the exploding gradient problem in RNNs/LSTMs
			nn.utils.clip_grad_norm_(model.parameters(), clip)

			# update
			optimizer.step()

			# compare
			predicted = (F.sigmoid(y_pred) > threshold).int()

			n = images.shape[0]
			y_true[start:start+n] = labels.cpu().numpy()
			y_preds[start:start+n] = predicted.cpu().numpy()
			start = start + n

			train_loss += loss.item()

		train_losses.append(train_loss / len(train_dataloader))
		sk_train_accs.append(accuracy_score(y_true, y_preds))
		sk_train_precisions.append(precision_score(y_true, y_preds, average="macro", zero_division=0.0))
		sk_train_recalls.append(recall_score(y_true, y_preds, average="macro", zero_division=0.0))
		sk_train_f1_scores.append(f1_score(y_true, y_preds, average="macro", zero_division=0.0))
		print("Epoch {:d}, Train Loss: {:.9f}, Train Accuracy: {:.7f}%".format(epoch+1, 
																			train_losses[-1], 
																			accuracy_score(y_true, y_preds)*100), end="")
		
		# initialize the hidden state
		val_hidden = model.lstm_model.init_hidden(batch_size=BATCH_SIZE)

		start = 0 # index
		y_true = np.zeros((val_data.shape[0], n_classes))
		y_preds = np.zeros((val_data.shape[0], n_classes))
		model.eval()
		for (_, images, captions, labels) in val_dataloader:

			# creating new variables for the hidden state, otherwise
			# we'd backprop through the entire training history
			val_hidden = tuple([each.data for each in val_hidden])
			
			if torch.cuda.is_available():
				images = images.to('cuda')
				captions = captions.to('cuda')
				labels = labels.to('cuda')
			
			outputs, _ = model(images, captions, val_hidden)
			predicted = (F.sigmoid(outputs) > threshold).int()

			n = images.shape[0]
			y_true[start:start+n] = labels.cpu().numpy()
			y_preds[start:start+n] = predicted.cpu().numpy()
			start = start + n
		
		sk_val_accs.append(accuracy_score(y_true, y_preds))
		sk_val_precisions.append(precision_score(y_true, y_preds, average="macro", zero_division=0.0))
		sk_val_recalls.append(recall_score(y_true, y_preds, average="macro", zero_division=0.0))
		sk_val_f1_scores.append(f1_score(y_true, y_preds, average="macro", zero_division=0.0))
		print(", Val Accuracy: {:.7f}%".format(accuracy_score(y_true, y_preds)*100))

	print(sk_train_accs)
	print(sk_train_precisions)
	print(sk_train_recalls)
	print(sk_train_f1_scores)
	print(sk_val_accs)
	print(sk_val_precisions)
	print(sk_val_recalls)
	print(sk_val_f1_scores)

	# create a dictionary
	results = {
		'train_accs': sk_train_accs,
		'train_precisions': sk_train_precisions,
		'train_recalls': sk_train_recalls,
		'train_f1_scores': sk_train_f1_scores,
		'val_accs': sk_val_accs,
		'val_precisions': sk_val_precisions,
		'val_recalls': sk_val_recalls,
		'val_f1_scores': sk_val_f1_scores,
	}

	return (model, results)

## Hyperparameter Analysis (Threshold)

In [12]:
EPOCHS = 50
N_CLASSES = 19

thresholds = [0.3, 0.5, 0.7, 0.9]
model_names = [
	"combined_threshold_0_3",
	"combined_threshold_0_5",
	"combined_threshold_0_7",
	"combined_threshold_0_9",
]
for threshold, model_name in zip(thresholds, model_names):
	print(threshold)
	print(model_name)

	model = CombinedModel(
		choice="regnet_x_1_6gf",
		pretrained=True, 		# if pretrained is False, then freeze should also be False
		freeze=True,
		cnn_n_out=256,
		no_layers=2,
		vocab_size=len(vocab), 	# already added by 1
		embedding_dim=64,
		lstm_hidden_dim=256,
		lstm_n_out=128,
		lstm_dropout=0.3,
		fc_dropout=0.5,
	)
	# utilise GPU
	if torch.cuda.is_available():
		print('using GPU')
		model = model.to('cuda')

	loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
	optimizer = torch.optim.Adam(
		params=model.parameters(),
		lr=0.001,
	)

	model, results = train_and_val(
		model=model,
		epochs=EPOCHS,
		n_classes=N_CLASSES,
		optimizer=optimizer,
		loss_fn=loss_fn,
		threshold=threshold,
		batch_size=BATCH_SIZE
	)
	torch.save(model.state_dict(), '../saved_models/' + model_name)
	results = pd.DataFrame(results)
	results.to_csv(f"../logs/{model_name}.csv", index=False)
	

0.3
combined_threshold_0_3
using GPU


Epoch 1 Training: 100%|██████████| 797/797 [00:52<00:00, 15.19it/s]


Epoch 1, Train Loss: 0.762590443, Train Accuracy: 0.5102041%, Val Accuracy: 1.0989011%


Epoch 2 Training: 100%|██████████| 797/797 [00:51<00:00, 15.47it/s]


Epoch 2, Train Loss: 0.665465513, Train Accuracy: 1.6483516%, Val Accuracy: 5.1491366%


...

Epoch 48, Train Loss: 0.079872340, Train Accuracy: 61.0753532%, Val Accuracy: 37.6138148%


Epoch 49 Training: 100%|██████████| 797/797 [00:50<00:00, 15.69it/s]


Epoch 49, Train Loss: 0.074987727, Train Accuracy: 61.6012559%, Val Accuracy: 36.9544741%


Epoch 50 Training: 100%|██████████| 797/797 [00:50<00:00, 15.73it/s]


Epoch 50, Train Loss: 0.069468310, Train Accuracy: 63.3751962%, Val Accuracy: 37.0486656%
[0.1695447409733124, 0.2551805337519623, 0.2820251177394035, 0.2991365777080063, 0.3120879120879121, 0.32221350078492933, 0.3291993720565149, 0.34419152276295134, 0.36475667189952904, 0.3693877551020408, 0.37676609105180536, 0.39050235478806905, 0.40800627943485085, 0.4153061224489796, 0.4318681318681319, 0.4310047095761381, 0.44599686028257457, 0.4503139717425432, 0.45729984301412874, 0.4684458398744113, 0.4785714285714286, 0.4803767660910518, 0.4971742543171115, 0.4993720565149137, 0.5104395604395604, 0.510989010989011, 0.5134222919937206, 0.5164835164835165, 0.5248822605965463, 0.5378335949764521, 0.5414442700156986, 0.5416012558869702, 0.5466248037676609, 0.548116169544741, 0.5585557299843014, 0.5677394034536891, 0.5729199372056515, 0.5733908948194663, 0.5719780219780219, 0.5842229199372057, 0.5896389324960754, 0.5889324960753533, 0.5944270015698587, 0.6010989010989011, 0.6112244897959184, 0.6

## Hyperparameter Analysis (Dropout Rate at Fully Connected Layer)

In [13]:
EPOCHS = 50
N_CLASSES = 19
THRESHOLD = 0.5

dropout_rates = [0.2, 0.4, 0.6, 0.8]
model_names = [
	"combined_dropout_0_2",
	"combined_dropout_0_4",
	"combined_dropout_0_6",
	"combined_dropout_0_8",
]
for dropout_rate, model_name in zip(dropout_rates, model_names):
	print(dropout_rate)
	print(model_name)

	model = CombinedModel(
		choice="regnet_x_1_6gf",
		pretrained=True, 		# if pretrained is False, then freeze should also be False
		freeze=True,
		cnn_n_out=256,
		no_layers=2,
		vocab_size=len(vocab), 	# already added by 1
		embedding_dim=64,
		lstm_hidden_dim=256,
		lstm_n_out=128,
		lstm_dropout=0.3,
		fc_dropout=dropout_rate,
	)
	# utilise GPU
	if torch.cuda.is_available():
		print('using GPU')
		model = model.to('cuda')

	loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
	optimizer = torch.optim.Adam(
		params=model.parameters(),
		lr=0.001,
	)

	model, results = train_and_val(
		model=model,
		epochs=EPOCHS,
		n_classes=N_CLASSES,
		optimizer=optimizer,
		loss_fn=loss_fn,
		threshold=THRESHOLD,
		batch_size=BATCH_SIZE
	)
	torch.save(model.state_dict(), '../saved_models/' + model_name)
	results = pd.DataFrame(results)
	results.to_csv(f"../logs/{model_name}.csv", index=False)

0.2
combined_dropout_0_2
using GPU


Epoch 1 Training: 100%|██████████| 797/797 [00:50<00:00, 15.80it/s]


Epoch 1, Train Loss: 0.716950691, Train Accuracy: 6.3343799%, Val Accuracy: 14.0659341%


Epoch 2 Training: 100%|██████████| 797/797 [00:50<00:00, 15.72it/s]


Epoch 2, Train Loss: 0.574437652, Train Accuracy: 13.2967033%, Val Accuracy: 19.4034537%


...

Epoch 48, Train Loss: 0.210662780, Train Accuracy: 40.7613815%, Val Accuracy: 33.3124019%


Epoch 49 Training: 100%|██████████| 797/797 [00:51<00:00, 15.51it/s]


Epoch 49, Train Loss: 0.214129025, Train Accuracy: 41.0282575%, Val Accuracy: 32.7158556%


Epoch 50 Training: 100%|██████████| 797/797 [00:51<00:00, 15.50it/s]


Epoch 50, Train Loss: 0.208197825, Train Accuracy: 40.9026688%, Val Accuracy: 33.5949765%
[0.015306122448979591, 0.03759811616954474, 0.053296703296703295, 0.07009419152276296, 0.08124018838304553, 0.09395604395604396, 0.10062794348508634, 0.108712715855573, 0.1205651491365777, 0.13084772370486655, 0.1403453689167975, 0.14607535321821036, 0.158712715855573, 0.16648351648351647, 0.186734693877551, 0.19309262166405022, 0.21224489795918366, 0.21773940345368917, 0.2270800627943485, 0.23383045525902668, 0.2401883830455259, 0.2529827315541601, 0.2627943485086342, 0.2729199372056515, 0.280690737833595, 0.2895604395604396, 0.29529042386185245, 0.30604395604395607, 0.31601255886970175, 0.32142857142857145, 0.32339089481946626, 0.3332810047095761, 0.3426216640502355, 0.34073783359497645, 0.33547880690737836, 0.3529827315541601, 0.35698587127158554, 0.3626373626373626, 0.3629513343799058, 0.3702511773940345, 0.38006279434850865, 0.3769230769230769, 0.37872841444270017, 0.391287284144427, 0.400392

## Hyperparameter Analysis (Learning Rate)

In [14]:
EPOCHS = 50
N_CLASSES = 19
THRESHOLD = 0.5

learning_rates = [0.01, 0.005, 0.001]
model_names = [
	"combined_lr_0_01",
	"combined_lr_0_005",
	"combined_lr_0_001",
]
for lr, model_name in zip(learning_rates, model_names):
	print(lr)
	print(model_name)
	
	model = CombinedModel(
		choice="regnet_x_1_6gf",
		pretrained=True, 		# if pretrained is False, then freeze should also be False
		freeze=True,
		cnn_n_out=256,
		no_layers=2,
		vocab_size=len(vocab), 	# already added by 1
		embedding_dim=64,
		lstm_hidden_dim=256,
		lstm_n_out=128,
		lstm_dropout=0.3,
		fc_dropout=0.5,
	)
	# utilise GPU
	if torch.cuda.is_available():
		print('using GPU')
		model = model.to('cuda')
	
	loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
	optimizer = torch.optim.Adam(
		params=model.parameters(),
		lr=lr,
	)

	model, results = train_and_val(
		model=model,
		epochs=EPOCHS,
		n_classes=N_CLASSES,
		optimizer=optimizer,
		loss_fn=loss_fn,
		threshold=THRESHOLD,
		batch_size=BATCH_SIZE
	)
	torch.save(model.state_dict(), '../saved_models/' + model_name)
	results = pd.DataFrame(results)
	results.to_csv(f"../logs/{model_name}.csv", index=False)
	

0.01
combined_lr_0_01
using GPU


Epoch 1 Training: 100%|██████████| 797/797 [00:50<00:00, 15.90it/s]


Epoch 1, Train Loss: 3.087514913, Train Accuracy: 1.1616954%, Val Accuracy: 4.7723705%


Epoch 2 Training: 100%|██████████| 797/797 [00:50<00:00, 15.84it/s]


Epoch 2, Train Loss: 3.500149600, Train Accuracy: 1.6954474%, Val Accuracy: 1.5698587%


...

Epoch 48, Train Loss: 0.068435336, Train Accuracy: 68.6656201%, Val Accuracy: 34.9450549%


Epoch 49 Training: 100%|██████████| 797/797 [00:51<00:00, 15.44it/s]


Epoch 49, Train Loss: 0.062643739, Train Accuracy: 70.7142857%, Val Accuracy: 35.8241758%


Epoch 50 Training: 100%|██████████| 797/797 [00:51<00:00, 15.52it/s]


Epoch 50, Train Loss: 0.071959969, Train Accuracy: 68.8069074%, Val Accuracy: 35.9183673%
[0.0630298273155416, 0.1270800627943485, 0.17189952904238617, 0.18838304552590268, 0.2098116169544741, 0.22346938775510203, 0.24175824175824176, 0.25502354788069076, 0.27237048665620095, 0.29725274725274725, 0.3076923076923077, 0.3361067503924647, 0.3540031397174254, 0.3700156985871272, 0.3904238618524333, 0.41208791208791207, 0.4328100470957614, 0.44199372056514913, 0.4589481946624804, 0.47543171114599686, 0.491287284144427, 0.490894819466248, 0.5117739403453689, 0.5217425431711146, 0.5347723704866562, 0.5350078492935636, 0.5594191522762951, 0.5683673469387756, 0.578021978021978, 0.5875196232339089, 0.5940345368916797, 0.591679748822606, 0.5999215070643642, 0.606828885400314, 0.610204081632653, 0.6204866562009419, 0.6288854003139718, 0.6232339089481946, 0.6278649921507065, 0.6500784929356358, 0.6642072213500785, 0.6661695447409733, 0.6504709576138148, 0.6557299843014128, 0.6707221350078493, 0.671